In [ ]:
# 練習 pandas 的 dataframe 資料調整和篩選方式

# 先載入資料並做成多日 dataframe

import os
import pandas

path = "D:\Python\爬蟲\動態爬蟲 ( Selenium )\三大法人每日買賣超資料抓取\download_20250805"

files = os.listdir(path)

all_data = []

for file in files : 

    with open(os.path.join(path,file),"r") as item : 

        dataframe = pandas.read_csv(item)

        dataframe["日期"] = file.split(".")[0]

        dataframe["日期"] = pandas.to_datetime(dataframe["日期"])

        dataframe["買賣差額"] = pandas.to_numeric(dataframe["買賣差額"].str.replace(",",""))

        dataframe = dataframe[["日期","單位名稱",'買賣差額']]        

        all_data.append(dataframe)

combine = pandas.concat(all_data)

pivot = combine.pivot(index="日期",columns="單位名稱",values="買賣差額")



# dataframe 的基本篩選方式


# 基本觀念 : 

# dataframe["欄位"] 代表該欄位底下的資料列

# dataframe["欄位"] > 0 代表該欄位底下的所有資料去做比較，並產生布林列

# dataframe[dataframe["欄位"] > 0] 表示對 dataframe 用布林列來篩選符合的列

# dataframe 基本上是二維的的表格，所以產生自己手動給定資料來做 dataframe 一定要用二維 list，不然一維 list 會變成一列一個元素

# 例如 : pandas.Dataframe([["20250801","aaa","120"]] , columns = column_list)

# 一維除非是用 pandas.series 才會正確，而不是 dataframe


# 1. 欄位 ( columns ) : 單純用 [] 和 欄位名稱 來做篩選即可

# a. 選擇單一欄位

combine_column = combine["單位名稱"]

print(combine_column)

# b. 選擇多個欄位 : 改用 list

combine_columns = combine[["單位名稱","買賣差額"]]

print(combine_columns)

# c. 移除單一欄位 : 用 drop 方法

combine_remove_column = combine.drop(columns="單位名稱")

print(combine_remove_column)

# d. 移除多個欄位 : 用 drop 方法 + list

combine_remove_columns = combine.drop(columns=["日期","單位名稱"])

print(combine_remove_columns)



# 2. 資料列 ( rows ) : 一樣用 [] , 但需要用欄位 + 條件運算子來產生布林列來挑出需要的資料列

# a. 日期挑選 ( dataframe 要先做 to_datetime() , 且注意日期要用字串表示，而不是直接打日期 )

combine_date_row = combine[combine["日期"]>"2025-08-01"]

print(combine_date_row)

# b. 特定群組挑選 ( 等於要用 == , 因為是比較產生布林值列來篩選資料列 )

combine_group_row = combine[combine["單位名稱"]=="合計"]

print(combine_group_row)

# c. 特定值挑選 ( dataframe 要先做 to_numeric() )

combine_value_row = combine[combine["買賣差額"] > 0]

print(combine_value_row)

# d. 合併條件篩選 ( 要用小括號 和 & 運算子 )

combine_multi_row = combine[(combine["日期"]>"2025-08-01")&(combine["單位名稱"]=="合計")]

print(combine_multi_row)

# e. 合併條件篩選 ( 用 isin , 基本上就是指定多個群組 == [ ... , ... ] 的操作 , 只是不能這樣做所以用 isin )

# 條件篩選就是用條件運算子 , 像是 == , > , < , != ( 排除資料可以用 ) 等等 , 

# isin 也是條件運算子 , 可產生布林值 , 只是代替 == [ ... , ... ] 而已

combine_multi_isin_row = combine[(combine["日期"]>"2025-08-01")&(combine["單位名稱"].isin(["投信","合計"]))]

print(combine_multi_isin_row)



# pivot 的篩選方式


# pivot 的觀念 : 

# pivot 和 dataframe 不同 , dataframe 的就是欄位和底下欄位的資料列 , 

# pivot 是 dataframe 的一個欄位當作 x 軸 , 另一個當作 y 軸 , 再一個為數值表，所以操作會不太一樣 , 

# pivot 是由 index , columns 和 values 所組成 , 都以這三者作為篩選條件的操作變數為主


# 1. 欄位的篩選方式

# 和 dataframe 一樣 , 都是用 [] , 因為 pivot 也是由 dataframe 構成 , 但要注意新索引 , 新欄位是由舊的組成 , 和原 dataframe 不同

# a. 選擇單一欄位 : 

pivot_column = pivot["投信"]

print(pivot_column)

# b. 選擇多個欄位 : 

pivot_columns = pivot[["投信","合計"]]

print(pivot_columns)

# c. 移除單一欄位 : 

pivot_remove_column = pivot.drop(columns="合計")

print(pivot_remove_column)

# d. 移除多個欄位 : 

pivot_remove_columns = pivot.drop(columns=["投信","合計"])

print(pivot_remove_columns)


# 2. 資料列的篩選方式

# 和 dataframe 一樣，用 [] 放 " 篩選條件產生的布林列 " 來挑選符合條件的資料列


# a. 用 index 來篩選資料列 ( 主要 )

pivot_index_row = pivot["2025-08-01":"2025-08-05"]

pivot_index_row_2 = pivot[pivot.index > "2025-08-01"]

print(pivot_index_row)

print(pivot_index_row_2)

# b. 沒有用 columns 來篩選資料列，但可以用 ["欄位"] > 0 這樣篩選某欄位下面資料列符合條件的資料

# ( 其他欄位還是會顯示，只有不符合資料列會消失，基本上還是資料列的篩選操作 )

pivot_column_row = pivot[pivot["合計"] > 0]

print(pivot_column_row)

# c. 沒有用 values 來篩選資料，因為 values 是二維數值矩陣，產生的布林值也是矩陣，沒辦法給 [] 只有一列資料列的陣列使用


# 3. 用 loc[] 函數來同時篩選欄位和資料列

# 兩個參數，第一個是列的布林陣列，放資料列的篩選條件，第二個是欄位的布林陣列，放欄位篩選條件，資料列和欄位的布林陣列都有

# ( 如果只做資料列可以只放一個參數 [] ，只做欄位就要用切片 [:, ] )

# 參數可以是單一標籤，也可以是多個標籤，切片，篩選條件 ( 索引也可以，因為本質還是 numpy array，例如 [:,0] , [0:1,1] )

# 適用 dataframe 也適用 pivot , 但是兩者欄位和資料列結構不同 , 所以要清楚自己用甚麼在做甚麼篩選，不要把欄位篩選產生的布林陣列放到資料列的


# a. dataframe 針對第一個參數的挑選 ( 只能用條件篩選 , 因為第一個參數挑選的是列 , 而 dataframe 挑列只能靠欄位做條件 , 且沒有日期當索引 )

dataframe_loc_row = combine.loc[combine["日期"] > "2025-08-01"]

print(dataframe_loc_row)

# b. dataframe 針對第二個參數的挑選 ( 基本上和原來一樣 , 就是 標籤 或 list )

dataframe_loc_column = combine.loc[:,"日期"]

dataframe_loc_column_2 = combine.loc[:,["單位名稱","買賣差額"]]

print(dataframe_loc_column)

print(dataframe_loc_column_2)

# c. pivot 針對第一個參數的挑選 ( 因為是列，所以就是用 index 的資料來操作，一樣有標籤或 list )

pivot_loc_row = pivot.loc["2025-08-01"]

pivot_loc_row_2 = pivot.loc[["2025-08-01","2025-08-05"]]

pivot_loc_row_3 = pivot.loc["2025-08-01":"2025-08-05"]

pivot_loc_row_4 = pivot.loc[pivot.index > "2025-08-01"]

print(pivot_loc_row)

print(pivot_loc_row_2)

print(pivot_loc_row_3)

print(pivot_loc_row_4)

# d. pivot 針對第二個參數的挑選 ( 因為是欄，所以就是用 columns 的資料來操作 )

pivot_loc_column = pivot.loc[:,"合計"]

pivot_loc_column_2 = pivot.loc[:,["投信","合計"]]

pivot_loc_column_3 = pivot.loc[:,pivot.columns.isin(["投信","合計"])]

print(pivot_loc_column)

print(pivot_loc_column_2)

print(pivot_loc_column_3)

# e. dataframe 和 pivot 的多條件篩選 

# ( 如果要做很多條件篩選就用 dataframe 多做幾次 loc 即可，另外也比較推薦先在 dataframe 做條件篩選再做 pivot，因為像 values 在 pivot 就沒辦法篩選，但在 dataframe 就比較好篩選 ) 

dataframe_loc_multi = combine.loc[(combine["日期"]>"2025-08-01")&(combine["買賣差額"]>0),["日期","單位名稱"]]

print(dataframe_loc_multi)

pivot_loc_multi = pivot.loc[pivot.index>"2025-08-01",pivot.columns.isin(["投信","合計"])]

print(pivot_loc_multi)



0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
Name: 單位名稱, dtype: object
             單位名稱         買賣差額
0       自營商(自行買賣)   1316827555
1         自營商(避險)   4056939563
2              投信   1437205201
3  外資及陸資(不含外資自營商)  14112204090
4           外資自營商            0
5              合計  20923176409
0       自營商(自行買賣)   2796495944
1         自營商(避險)   5602641455
2              投信     17825528
3  外資及陸資(不含外資自營商)   9388571808
4           外資自營商            0
5 

In [ ]:
# dataframe 基本觀念補充


import pandas
import numpy


# 1. dataframe 不能直接當 list 做 for 迴圈的 iterable 的讀取，會產生錯誤結果

data_column = ["日期","名稱","收盤價"]

data_row = [["20250801","aaa",120],["20250802","aaa",float("Nan")],["20250803","aaa", ],["20250804","   ",150],[None,"aaa",160]]

dataframe = pandas.DataFrame(data_row,columns=data_column)

for row in dataframe : 

    print(row)

# numpy array 是可以的，因為是用 list 當基礎

array = numpy.array([[1,2,3],[4,5,6],[7,8,9]])

for row in array : 

    print(row)

# pandas dataframe 還是得要靠切片或 loc / iloc 來讀取每列資料 , 但基本上會比較推薦直接用 dataframe["..."] 的欄位向量化操作，會比較快


# 兩種函數方式取每列資料 : 

# 可以用 range(len(df)) 或 df.index 來當迴圈變數


# a. iterrows() : 

# 最常見 , 但效能比較不好

# 如果直接全部欄位放下去，像是 row[["日期","名稱","收盤價"]]，讀取會變成詳細資料格式，比較不好用，要分開

for index , row in dataframe.iterrows() : 

    print(index , row["日期"] , row["名稱"] , row["收盤價"])


# b. itertuples() : 

# 速度比較快

# 回傳 : namedtuple ( 就是有 name 的 tuple , 基本上還是一樣用法 , 用 ["索引值"] 來取得值 , 但多了用屬性取得值 )

for row in dataframe.itertuples() : 

    print(row)

for row in dataframe.itertuples() : 

    print(row[0] , row[1] , row[2] , row[3])

# 如果用屬性來得到值 , 記得不要用 index 屬性 , 會是 namedtuple 的原生屬性 , 變成其他物件

for row in dataframe.itertuples() : 

    print(row[0] , row.日期 , row.名稱 , row.收盤價)

# 回傳 : 一般 tuple

for row in dataframe.itertuples(name=None) : 

    index , date , name , close_price = row

    print(index , date , name , close_price)


# c. iterrows() , itertuples() , df["..."] , iloc/loc() , apply() , df.values 幾種方式讀取列比較 : 

# df["..."] : 最快 , 是向量化作法

# itertuples() : 第二快 , 因為使用接近原生 tuple 的 namedtuple , 但只做讀取

# df.values : 第三快 , 因為轉成 numpy array , 純數值處理 ( 這邊比較特別 , 因為是 pandas 轉換的矩陣 , 所以 dtype 是混和型 object 而不會直接轉成字串 U21, 元素還是保有原來型態 )

# for + iloc() / loc() : 第四快 , 因為是每列讀取 , 而且還要呼叫 iloc() / loc() 函數來讀取資料

# apply() : 第五快 , 類似向量化 , apply 是一列一列或一欄一欄處理 , 比 for 迴圈快 , 適合需要 if 使用

# iterrows() : 最慢 , 因為是把每列轉成 series 來處理 , 比較適合少量資料



# 2. dataframe 的元素長度

# dataframe 的長度是資料列的多寡 . 和 numpy 這點就一樣

print(len(dataframe))

print(len(array))



# 3. dataframe 的切片

# dataframe 沒有切片 , 不像 numpy array , 可以直接 array[1,:]

# 要用 dataframe 專屬的 iloc / loc 函數 , 這基本上就是 dataframe 的切片

print(dataframe.iloc[:,2])

print(array[:,2])



# 4. pandas 的 series

# 定義上是 1D 的 pandas 陣列，只有一列資料，像是一個 1D array，2D 是 dataframe

# 實際上可以發現是看取欄或取列資料，再以欄或列為欄位名稱的序列 , 就是短的那邊為欄 , 長的那邊為列 , 這樣才像一個序列 , 可以看下面例子

# 欄的序列 : df["..."] , 以欄作為欄位名稱 , 列變成 index 

column_series = dataframe["收盤價"]

print(column_series)

# 列的序列 : df.iloc[0] , 以列作為欄位名稱 , 欄變成 index

row_series = dataframe.iloc[0]

print(row_series)

# 技巧 : 如何維持 2D 以利後續加入到原來 dataframe

# 欄的序列 : df[["..."]] 兩個中括號會是只有一個序列的 2D 的 dataframe 

column_dataframe = dataframe[["收盤價"]]

print(column_dataframe)

# # 列的序列 : 也是雙重中括號 iloc[["..."]] , 或者 to_frame() 轉成 2D 再做 T 轉置

row_dataframe = dataframe.iloc[[0]]

print(row_dataframe)



日期
名稱
收盤價
[1 2 3]
[4 5 6]
[7 8 9]
0 20250801 aaa 120.0
1 20250802 aaa nan
2 20250803 aaa nan
3 20250804     150.0
4 None aaa 160.0
Pandas(Index=0, 日期='20250801', 名稱='aaa', 收盤價=120.0)
Pandas(Index=1, 日期='20250802', 名稱='aaa', 收盤價=nan)
Pandas(Index=2, 日期='20250803', 名稱='aaa', 收盤價=nan)
Pandas(Index=3, 日期='20250804', 名稱='   ', 收盤價=150.0)
Pandas(Index=4, 日期=None, 名稱='aaa', 收盤價=160.0)
0 20250801 aaa 120.0
1 20250802 aaa nan
2 20250803 aaa nan
3 20250804     150.0
4 None aaa 160.0
0 20250801 aaa 120.0
1 20250802 aaa nan
2 20250803 aaa nan
3 20250804     150.0
4 None aaa 160.0
0 20250801 aaa 120.0
1 20250802 aaa nan
2 20250803 aaa nan
3 20250804     150.0
4 None aaa 160.0
5
3
0    120.0
1      NaN
2      NaN
3    150.0
4    160.0
Name: 收盤價, dtype: float64
[3 6 9]
0    120.0
1      NaN
2      NaN
3    150.0
4    160.0
Name: 收盤價, dtype: float64
日期     20250801
名稱          aaa
收盤價       120.0
Name: 0, dtype: object
     收盤價
0  120.0
1    NaN
2    NaN
3  150.0
4  160.0
         日期   名稱    收盤價
0  20

In [15]:
# pandas 其他技巧



# 1. dataframe 和 pivot 一樣都是 dataframe , 只是行和列定義不太一樣，但都有 dataframe.index , dataframe.columns , dataframe.values 

# 可以對 dataframe.columns 和 dataframe.values 做 list , 就能得到欄位名稱和資料的 list 

# ( dataframe 的 dataframe.index 就只是索引，沒甚麼用，pivot 是另一個欄位來當索引，所以 pivot 用 pivot.index 比較有用 )

import pymysql
import pandas

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    database="example",
    password="12345",
    charset="utf8"

)

sql = "SELECT * FROM new_table_2"

result = pandas.read_sql(sql,connection)

print(list(result.columns))

print(list(result.values))



# 2. dataframe 可以用 dataframe.to_numpy 轉成 numpy，有些使用場合會比較方便

# result 原本是 dataframe

print(result)

# 用 dataframe.to_numpy 轉成 numpy，但是 numpy array 沒有 column 和 row label，所以欄位名稱和列索引會不見

result_2 = result.to_numpy()

print(result_2)


C:\Users\zxzxa\AppData\Local\Temp\ipykernel_11936\140349258.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pandas.read_sql(sql,connection)


['Seq', '年度', '國小補校校數', '國中補校校數', '高級中等學校進修部校數', '國小補校班級數', '國中補校班級數', '高級中等學校進修部班級數']
[array(['1', '2017', '15', '24', '17', '23', '44', '172'], dtype=object), array(['2', '2018', '15', '23', '17', '22', '43', '165'], dtype=object), array(['3', '2019', '16', '22', '17', '24', '36', '155'], dtype=object), array(['4', '2020', '15', '20', '15', '23', '35', '133'], dtype=object), array(['5', '2021', '15', '17', '15', '23', '32', '128'], dtype=object), array(['6', '2022', '14', '16', '14', '20', '29', '120'], dtype=object), array(['7', '2023', '13', '16', '14', '20', '29', '114'], dtype=object), array(['8', '2024', '13', '16', '14', '18', '28', '105'], dtype=object)]
  Seq    年度 國小補校校數 國中補校校數 高級中等學校進修部校數 國小補校班級數 國中補校班級數 高級中等學校進修部班級數
0   1  2017     15     24          17      23      44          172
1   2  2018     15     23          17      22      43          165
2   3  2019     16     22          17      24      36          155
3   4  2020     15     20          15      23      35       

In [14]:
# pandas 的 concatenate 函數


# pandas 的 dataframe 也算是一種陣列，因此也有像 numpy 的 concatenate 的函數，來方便做 dataframe 的合併

# pandas 比較單純，就只有 concatenate，沒有 stack，但是 pandas 的 concatnate 和 numpy 的版本一樣，

# 也有分 axis = 0 or 1 不同方向的疊加，且 pandas 的 concatenate 也沒有增加維度

# 注意 : dataframe 建立時欄位名稱沒給沒關係，會變成像 numpy 是用索引，不過一定要用 2D 表達，不然格是會變很奇怪，每一列會變成直接垂直一直往下放


import pandas


# 一維的例子

column = ["日期","名稱","收盤價"]

# 單列資料務必要用二維的方式來表達，不然 pandas 會以為是 n 筆資料分成不同列

dataframe_1 = pandas.DataFrame([["2025/08/01","aaa",100]],columns=column)
dataframe_2 = pandas.DataFrame([["2025/08/02","bbb",110]],columns=column)
dataframe_3 = pandas.DataFrame([["2025/08/03","ccc",120]],columns=column)

# axis = 0 ( 向 y 軸延伸，預設 )

combine_1 = pandas.concat([dataframe_1,dataframe_2,dataframe_3])

print(combine_1)

# axis = 1 ( 向 x 軸延伸)

combine_2 = pandas.concat([dataframe_1,dataframe_2,dataframe_3],axis=1)

print(combine_2)


# 二維的例子

dataframe_4 = pandas.DataFrame([["2025/08/04","ddd",125],["2025/08/05","ddd",130]],columns=column)
dataframe_5 = pandas.DataFrame([["2025/08/06","eee",135],["2025/08/07","eee",140]],columns=column)

# axis = 0 ( 向 y 軸延伸，預設 )

combine_3 = pandas.concat([dataframe_4,dataframe_5])

print(combine_3)

# axis = 1 ( 向 x 軸延伸)

combine_4 = pandas.concat([dataframe_4,dataframe_5],axis=1)

print(combine_4)



           日期   名稱  收盤價
0  2025/08/01  aaa  100
0  2025/08/02  bbb  110
0  2025/08/03  ccc  120
           日期   名稱  收盤價          日期   名稱  收盤價          日期   名稱  收盤價
0  2025/08/01  aaa  100  2025/08/02  bbb  110  2025/08/03  ccc  120
           日期   名稱  收盤價
0  2025/08/04  ddd  125
1  2025/08/05  ddd  130
0  2025/08/06  eee  135
1  2025/08/07  eee  140
           日期   名稱  收盤價          日期   名稱  收盤價
0  2025/08/04  ddd  125  2025/08/06  eee  135
1  2025/08/05  ddd  130  2025/08/07  eee  140


In [ ]:
# pandas 的相關函數


import pandas


# 1. insert() 

# pandas 有 insert 可以新增欄位，跟 pandas["..."] 直接產生新欄位一樣

data_column = ["日期","名稱","收盤價"]

# 資料記得要用 2D 陣列 [[...] , [...] , ...] 

data_row = [["20250801","aaa",120]]

dataframe = pandas.DataFrame(data_row,columns=data_column)

# 用 insert 新增欄位，比較方便的是可以指定第幾欄，第一個引數是要插入的位置，第二個是欄位名稱，第三個是資料

dataframe.insert(2,column="開盤價",value=110)

print(dataframe)



# 2. append()

# dataframe 如果要新增一筆資料，也就是合併一個 series，可以用 append() 

# 新增多筆資料，也就是合併一個 dataframe，就用 concat()



# 3. copy() 

# 就跟 numpy array 一樣，pandas 要複製 dataframe 可以用 copy()



# 4. shape

# pandas 也有 shape 可以用，和 numpy 一樣，可以方便取得列和欄的數量

array = dataframe_3.shape

print(array)

# 一樣可以用索引直接取得回傳的 tuple 的列和欄總數

print(array[0])

print(array[1])



# 5. loc() 和 iloc()

# loc 和 iloc 差別就像之前所說，loc 是針對文字標籤進行篩選的函數，iloc 是利用 index 數字標籤去篩選

# 不過 loc 其實也支援 index 數字標籤篩選，但前提是還沒有給定文字標籤之前，基本上就是看 dataframe 的索引型態來決定

# loc 和 iloc 用數字索引標籤時，注意是包含最後的一個數字，不像 range 或 一般 list 的切片會省略最後一個數字


# a. 刻意不給欄位文字標籤，這樣就可以用數字索引篩選

data_row_2 = [["20250801","aaa",120],["20250802","aaa",130],["20250803","aaa",140],["20250804","aaa",150],["20250805","aaa",160]]

dataframe_2 = pandas.DataFrame(data_row_2)

select = dataframe_2.loc[0:2,0:1]

print(select)

# 改成欄位有文字標籤，就不能對欄位用數字索引，只能對數字索引的資料列做篩選

dataframe_3 = pandas.DataFrame(data_row_2,columns=data_column)

# select = dataframe_2.loc[0:2,0:1]

select_2 = dataframe_2.loc[0:2]

print(select_2)


# b. 不過要注意，有 concatenate 過的 dataframe 會出錯，因為索引會重複，可以看下面 concat 的結果

# 因此盡量還是 loc 用文字，iloc 用數字，這樣比較不會有問題

combine = pandas.concat([dataframe_3,dataframe_3])

print(combine)

# 用 loc 會錯誤，顯示有重複索引

# select_3 = combine.loc[0:2]

# print(select_3)

# iloc 就會正確，不會管數字標籤有重複

select_4 = combine.iloc[0:6]

print(select_4)







         日期   名稱  開盤價  收盤價
0  20250801  aaa  110  120
          0    1
0  20250801  aaa
1  20250802  aaa
2  20250803  aaa
          0    1    2
0  20250801  aaa  120
1  20250802  aaa  130
2  20250803  aaa  140
         日期   名稱  收盤價
0  20250801  aaa  120
1  20250802  aaa  130
2  20250803  aaa  140
3  20250804  aaa  150
4  20250805  aaa  160
0  20250801  aaa  120
1  20250802  aaa  130
2  20250803  aaa  140
3  20250804  aaa  150
4  20250805  aaa  160
         日期   名稱  收盤價
0  20250801  aaa  120
1  20250802  aaa  130
2  20250803  aaa  140
3  20250804  aaa  150
4  20250805  aaa  160
0  20250801  aaa  120
(5, 3)
5
3


In [2]:
# pandas 資料型別

import pandas



# pandas 和 numpy 一樣 , 只要一欄有一個資料是文字 , 剩下都會轉成文字

data_column = ["日期","名稱","收盤價"]

data_row = [["20250801","aaa",120],["20250802","aaa","130"],["20250803","aaa",140]]

dataframe = pandas.DataFrame(data_row,columns=data_column)

# 直接看 dataframe 看不出來

print(dataframe)

# 要用 info 看各欄位 dtype

dataframe.info()


# 改成數字全部一致就正確了

data_row_2 = [["20250801","aaa",120],["20250802","aaa",130],["20250803","aaa",140]]

dataframe_2 = pandas.DataFrame(data_row_2,columns=data_column)

print(dataframe_2)

dataframe_2.info()



         日期   名稱  收盤價
0  20250801  aaa  120
1  20250802  aaa  130
2  20250803  aaa  140
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   日期      3 non-null      object
 1   名稱      3 non-null      object
 2   收盤價     3 non-null      object
dtypes: object(3)
memory usage: 204.0+ bytes
         日期   名稱  收盤價
0  20250801  aaa  120
1  20250802  aaa  130
2  20250803  aaa  140
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   日期      3 non-null      object
 1   名稱      3 non-null      object
 2   收盤價     3 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes
